# Segmenting and Clustering Neighborhoods in Toronto

In [35]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge geopy
#!pip install geocoder
#!conda install -c conda-forge folium=0.5.0 --yes 
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries Imported')

Libraries Imported


# HTML Parsing & Data Pre-Processing
### - Using pandas for parsing HTML & Data Wrangling

In [2]:
# parse HTML and get the first occurence of <table> tag
canada_neighborhood_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]

# remove Borough with 'Not Assigned' value & reset index
canada_neighborhood_df = canada_neighborhood_df[canada_neighborhood_df.Borough != 'Not assigned'].reset_index(drop=True)

# rename column : Postal code->PostalCode
canada_neighborhood_df.rename(columns = {'Postal code':'PostalCode'}, inplace = True) 

# replace "/" with ","
temp_df= canada_neighborhood_df['Neighborhood']
temp_df = temp_df.replace(r' /', ',', regex=True)
temp_df = temp_df.replace(r'\'', '', regex=True)
canada_neighborhood_df['Neighborhood']=temp_df

# sort dataframe for readability
canada_neighborhood_df = canada_neighborhood_df.sort_values('PostalCode').reset_index(drop=True)
canada_neighborhood_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Canada Neighborhood Data Frame Shape

In [3]:
canada_neighborhood_df.shape

(103, 3)

# Method # 1 : Use  google geocoder API For Determining Lat/Long [with Postal Codes]

### Assumptions
<ol>
<li>Use google geocoder API</li>
<li>Use postal code to determine Lat/Long</li>
<li>Attempt multiple retries</li>
</ol>

In [4]:
import geocoder # import geocoder

# define the dataframe columns
column_names_geocoder = ['Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods_geocoder = pd.DataFrame(columns=column_names_geocoder)

for idx_geo in canada_neighborhood_df.index:
    postal_code = canada_neighborhood_df['PostalCode'][idx_geo]
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        
    lat_lng_coords = g.latlng
    
    if(lat_lng_coords != None):
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        print('Postal Code :', postal_code, ' Lat/Long : ', latitude, longitude)
    else:
        latitude = 0.0
        longitude = 0.0

### Result :
<ol>
<li>Despite multiple retries, including using while loops, could not retrieve Lat/Long with Postal Codes</li>
<li>Attempt to derive Lat/Long using Nominatim geolocator [with Neighborhood name], as shown below in Method # 2</li>
</ol>

# Method # 2 : Use Nominatim geolocator API For Determining Lat/Long [with Neighborhood names]

### Assumptions
<ol>
<li>Use Nominatim geolocator API</li>
<li>Use Neighborhood name to determine Lat/Long</li>
<li>Use delay, to avoid timeouts</li>
<li>Use Downtown Ontario Lat/Long as default, if the Lat/Long cannot be resoloved</li>
</ol>

In [5]:
# define the dataframe columns
column_names = ['Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Initialize
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="toronto_geo_explorer")

# Use default Ontario Downton Lat/Long, if the address cannot be found using Neighborhood address
lat_default = 43.6547567
lon_default = -79.3966769

for idx in canada_neighborhood_df.index:
    address = canada_neighborhood_df['Neighborhood'][idx]
    # for postal codes with multiple neighborhoods, take the first one
    address = address.split(',')
    address = address[0]
    address = address + ',ON'

    # add delay to avoid timeouts ..
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)
    
    location = geolocator.geocode(address)
    if(location != None):
        latitude = location.latitude
        longitude = location.longitude
        neighborhoods = neighborhoods.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
    else:
        # Address cannot be located by the API. Assign default address
        neighborhoods = neighborhoods.append({'Latitude': lat_default,'Longitude': lon_default}, ignore_index=True)

canada_neighborhood_df['Latitude'] = neighborhoods['Latitude']
canada_neighborhood_df['Longitude'] = neighborhoods['Longitude']

### Result :
<ol>
<li>with adding delays,  able to resolve timeout issue</li>
<li>Able to retrieve Lat/Long. Here is the output</li>
</ol>

In [6]:
canada_neighborhood_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221701
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.780271,-79.130499
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.755225,-79.198229
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692
5,M1J,Scarborough,Scarborough Village,43.743742,-79.211632
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.724878,-79.253969
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.727841,-79.287622
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.711170,-79.248177
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.691639,-79.266110


# Analyze Neighborhood's Within Downtown Toronto Borough

In [18]:
# Analyze Neighborhoods within Downtown Toronto Borough
toronto_neighborhood_df = canada_neighborhood_df[canada_neighborhood_df['Borough'].str.contains('Downtown Toronto')].reset_index(drop=True)
toronto_neighborhood_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.678356,-79.380746
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.669403,-79.372704
2,M4Y,Downtown Toronto,Church and Wellesley,43.665524,-79.383801
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.656500,-79.377114
5,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
6,M5E,Downtown Toronto,Berczy Park,43.647984,-79.375396
7,M5G,Downtown Toronto,Central Bay Street,22.283993,114.156639
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",39.828690,-84.889852
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640080,-79.380150


# Map Downtown Toronoto Neighborhoods

In [19]:
address = 'Downtown Toronto,ON'

geolocator = Nominatim(user_agent="toronto_downtown_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of DownTown are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of DownTown are 43.6563221, -79.3809161.


In [23]:
# create map of Downtown Toronto using latitude and longitude values
downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map highlighting each neighborhood
for lat, lng, label in zip(toronto_neighborhood_df['Latitude'], toronto_neighborhood_df['Longitude'], toronto_neighborhood_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(downtown_toronto)  
    
downtown_toronto

### Foursquare API client Id initialization

In [28]:
CLIENT_ID = 'UA0R0KYNOMC1QQBURRIUPMO3BXJDV5JPJBONEBQQBDG5FHMZ' # your Foursquare ID
CLIENT_SECRET = '2S41RJYWFK5HYEHH0J2XNILXAJOHXCHQJOBWV54HCXEHK0SX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UA0R0KYNOMC1QQBURRIUPMO3BXJDV5JPJBONEBQQBDG5FHMZ
CLIENT_SECRET:2S41RJYWFK5HYEHH0J2XNILXAJOHXCHQJOBWV54HCXEHK0SX


### Define Function to get Venues for a give Neighborhood DataFrame

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    radius = 500 # define radius
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
toronto_venues = getNearbyVenues(names=toronto_neighborhood_df['Neighborhood'],
                                   latitudes=toronto_neighborhood_df['Latitude'],
                                   longitudes=toronto_neighborhood_df['Longitude']
                                  )
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.678356,-79.380746,Alex Murray Parkette,43.678300,-79.382773,Park
1,Rosedale,43.678356,-79.380746,Betline Trail at Roxborough dr.,43.680530,-79.381490,Bike Trail
2,Rosedale,43.678356,-79.380746,Park Drive Reservation Lands,43.679822,-79.377787,Park
3,Rosedale,43.678356,-79.380746,Rosedale Park,43.682328,-79.378934,Playground
4,"St. James Town, Cabbagetown",43.669403,-79.372704,Mr. Jerk,43.667328,-79.373389,Caribbean Restaurant


In [41]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 218 uniques categories.


### Analyze Each Neighborhood

In [45]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Afghan Restaurant,Airport Service,American Restaurant,Animal Shelter,Aquarium,Argentinian Restaurant,Art Gallery,...,Theater,Theme Restaurant,Tour Provider,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.0000,0.00,0.010000,0.000000,0.000000,0.00,0.020000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.013333,0.000000,0.0000,0.00,0.000000,0.000000,0.026667,0.00,0.000000,...,0.013333,0.0000,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.013333
2,Central Bay Street,0.030000,0.000000,0.0000,0.01,0.000000,0.000000,0.000000,0.01,0.010000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.030000
3,Christie,0.000000,0.000000,0.0000,0.00,0.017241,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.017241,0.017241
4,Church and Wellesley,0.025000,0.000000,0.0125,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.00,0.040000,0.000000,0.000000,0.00,0.010000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.0000,0.00,0.040000,0.000000,0.000000,0.00,0.010000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000
7,"Garden District, Ryerson",0.000000,0.000000,0.0000,0.00,0.014085,0.000000,0.000000,0.00,0.000000,...,0.028169,0.0000,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.00,0.010000,...,0.010000,0.0000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.040000,0.010000


### Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Placing the grouped data in a dataframe

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Café,Bakery,Seafood Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Cocktail Bar,Beer Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Hotel,Pizza Place,Bar,Italian Restaurant,Restaurant,Baseball Stadium,Gym,Concert Hall,Ice Cream Shop
2,Central Bay Street,Japanese Restaurant,Gym / Fitness Center,Cantonese Restaurant,French Restaurant,Wine Bar,Cocktail Bar,Hotel,Italian Restaurant,Gym,Yoga Studio
3,Christie,Korean Restaurant,Coffee Shop,Dessert Shop,Sandwich Place,Karaoke Bar,Grocery Store,Ice Cream Shop,Café,Japanese Restaurant,Cocktail Bar
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Burger Joint,Sushi Restaurant,Restaurant,Yoga Studio,Hotel,Café,Gastropub


## Cluster Neighborhoods

In [53]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 4, 2, 0, 0, 0, 0, 2], dtype=int32)

In [54]:
# add clustering labels and create dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged_df = toronto_neighborhood_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged_df = toronto_merged_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_df.head() # check the last columns!

# add cluster labels
# toronto_merged_df['Cluster Description'] = {}

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.678356,-79.380746,1,Park,Bike Trail,Playground,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.669403,-79.372704,0,Coffee Shop,Pizza Place,Grocery Store,Café,Ice Cream Shop,Bistro,Market,Beer Store,Filipino Restaurant,Bike Rental / Bike Share
2,M4Y,Downtown Toronto,Church and Wellesley,43.665524,-79.383801,2,Coffee Shop,Japanese Restaurant,Gay Bar,Burger Joint,Sushi Restaurant,Restaurant,Yoga Studio,Hotel,Café,Gastropub
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457,0,Coffee Shop,Thai Restaurant,Restaurant,Beer Store,Indian Restaurant,Pharmacy,Pet Store,Performing Arts Venue,Sushi Restaurant,Auto Dealership
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.656500,-79.377114,0,Clothing Store,Restaurant,Coffee Shop,Middle Eastern Restaurant,Hotel,Thai Restaurant,Cosmetics Shop,Movie Theater,Japanese Restaurant,Ramen Restaurant


### Visualize Cluster Neighborhoods

In [59]:
# create map
toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_df['Latitude'], toronto_merged_df['Longitude'], toronto_merged_df['Neighborhood'], toronto_merged_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_clusters)
       
toronto_clusters

## Cluster # 1

<ol>
<li>Highlighted in Red Markers</li>
<li>Densely Clustered</li>
<li>Venues include Restaurants, Coffee Shops</li>
</ol>

## Cluster # 2

<ol>
<li>Highlighted in Blue Markers</li>
<li>Venues include Hotels, Yoga Studio</li>
</ol>

## Cluster # 3

<ol>
<li>Highlighted in Purple Markers</li>
<li>Venues include Parks, Bike Trails, Outdoor activities</li>
<li>Primarily on the outskirts of Toronto Downtown</li>
</ol>